In [2]:
import os
import pandas as pd
import re
import time
import fitz
import winsound
import concurrent.futures
from tqdm import tqdm
import cProfile
import traceback
import numpy as np
from process_file2 import process_file, process_file_wrapper
from multiprocessing import Pool, cpu_count
from multiprocessing import Value
from multiprocessing import Manager, Queue
from pdfrw import PdfReader, PdfWriter


pd.set_option('display.max_columns', None)

# specify the directory where the PDF files are stored
Directory = 'C:/Hanbo Wang/LEAP/2023/ELA/t'

# Initialize an empty DataFrame
Variable_List = ['If_Voided', 'Report_Title', 'Report_Subject', 'Report_Type', 'Report_Year','Report_Season',
                 'Report_Date', 'Student_Name', 'LASID', 'DoB', 'Grade', 'School', 'School_System', 'Overview',
                 'State_Average_Score', 'State_Average_Level',  'State_Average_Achievement_Level',
                 'Student_Performance_Score', 'Student_Performance_Level', 'Student_Performance_Achievement_Level',
                 'School_System_Average_Score', 'School_System_Average_Level', 'School_System_Average_Achievement_Level',
                'Overall_Student_Performance']


# Coordinate Dictionary
C_dict = {
    'If_Voided': (100.99799346923828, 240.17391967773438, 364.7249450683594, 304.9959716796875),
    'Report_Title_Section':(268.41949462890625, 24.466045379638672, 388.220458984375, 60.76609420776367),
    'Report_Title': (299.8905029296875, 24.466045379638672, 356.7494812011719, 35.46604537963867), 
    'Report_Subject': (307.530029296875, 37.11606979370117, 349.1100158691406, 48.11606979370117),
    'Report_Time':(297.1405029296875, 49.76609420776367, 359.4995422363281, 60.76609420776367),
    'Personal_Information': (36.0, 81.936432, 555.31427, 119.298805),
    'Overview': (36.0, 154.26290893554688, 573.7233276367188, 217.44131469726562),
    'Student_Performance_Score': (99.5, 272.1, 134.5139923095703, 304.9959716796875),
    'School_System_Average_Score': (437.9, 272.0903015136719, 472.9139709472656, 304.9959716796875),
    'State_Average_Score': (534.4, 272.0903015136719, 569.39404296875, 304.9959716796875),
    'Student_Performance_Level': (48.6, 276.0442199707031, 77.60700225830078, 306.5198974609375),
    'School_System_Average_Level': (392.8, 276.0442199707031, 421.7669982910156, 306.5198974609375),
    'State_Average_Level': (489.24, 276.0442199707031, 518.2470092773438, 306.5198974609375),
    'Overall_Student_Performance': (159.12, 240.17391967773438, 373.2570495605469, 311.8724365234375),
    'Achievement_Levels':(48.5, 313.1333312988281, 541.7593994140625, 332.9332275390625),
    'Achievement_Levels_Fixed':(48.5, 313.1333312988281, 561.5098876953125, 332.9332275390625),
    'Student_Achievement_Level':(48.5, 313.1333312988281, 138.4506072998047, 332.9332275390625),
    'School_System_Average_Achievement_Level':(431.37921142578125, 313.1333312988281, 475.8841552734375, 332.9332275390625),
    'State_Average_Achievement_Level':(522.6917724609375, 313.1333312988281, 567.1967163085938, 332.9332275390625),
    'State_Average_Achievement_Level_Fixed':(480, 313.1333312988281, 567.1967163085938, 332.9332275390625),
    'Investigate_Level': (47.3, 361.6931457519531, 109.95957946777344, 377.6131591796875),
    'Investigate_Rating': (121.3, 349.7764587402344, 565.6683959960938, 386.82415771484375),
    'Evaluate_Level': (47.3, 411.753173828125, 109.95957946777344, 427.6731872558594),
    'Evaluate_Level_Fixed':(47.3, 398, 109.9595718383789, 440.3182067871094),
    'Evaluate_Rating':(121.3, 399.83648681640625, 575.6884765625, 454.1341857910156),
    'Reason_Scientifically_Level': (47.3, 474.4582214355469, 109.95957946777344, 490.37823486328125),
    'Reason_Scientifically_Level_Fixed':(47.3, 468, 109.95957946777344, 521),
    'Reason_Scientifically_Rating': (121.3, 462.5415344238281, 572.3583984375, 499.5892333984375),
    'Percent_Achievement_Level_Advanced': (90.60675048828125, 669.9880981445312, 551.4429321289062, 678.0706176757812),
    'Percent_Achievement_Level_Mastery': (90.60675048828125, 683.6680908203125, 553.6669311523438, 691.6680908203125),
    'Percent_Achievement_Level_Basic': (90.60675048828125, 697.3480834960938, 553.6669311523438, 705.3480834960938),
    'Percent_Achievement_Level_Approaching_Basic': (90.60675048828125, 711.028076171875, 553.6669311523438, 719.028076171875),
    'Percent_Achievement_Level_Unsatisfactory': (90.60675048828125, 724.7080688476562, 553.6669311523438, 732.7080688476562),
    'Reading_Performance_Achievement_Level':(39.1879997253418, 387.57275390625, 94.01200103759766, 402.62274169921875),
    'Literary_Text_Achievement_Level':(39.57841873168945, 431.7209777832031, 94.40242004394531, 461.5955810546875),
    'Informational_Text_Achievement_Level':(39.57841491699219, 486.1009826660156, 94.40241241455078, 515.9755859375),
    'Vocabulary_Achievement_Level':(39.57841491699219, 540.4810180664062, 94.40241241455078, 570.3556518554688),
    'Reading_Performance_Achievement_Level_State_Percentages':(128.6118927001953, 407.6592102050781, 276.72808837890625, 417.1592102050781),
    'Writing_Performance_Achievement_Level':(317.1080017089844, 373.46807861328125, 371.9320373535156, 403.34271240234375),
    'Writing_Performance_Achievement_Level_State_Percentages':(404.37164306640625, 409.0992126464844, 552.4866943359375, 418.5992126464844),
    'Written_Expression_Achievement_Level':(317.33111572265625, 431.72100830078125, 372.15509033203125, 461.5956115722656),
    'Knowledge&Use_of_Language_Conventions':(317.33111572265625, 486.1009826660156, 372.15509033203125, 515.9755859375)
}


VS = pd.Series(Variable_List)

Subjects_in_Headlines = ['ELA']


Report_Type = 'StudentReport'



In [3]:
def combine_pdfs(Directory, Subjects_in_Headlines, Report_Type):
    
    start_time = time.time()
    
    # Get a list of all the PDF files in the directory
    pdf_files = [f for f in os.listdir(Directory) if f.endswith('.pdf') and any(Subject in f for Subject in Subjects_in_Headlines) and Report_Type in f]

    # Create a new PDF writer object
    writer = PdfWriter()

    # Iterate over the PDF files
    for filename in tqdm(pdf_files, desc="Combining PDFs"):
        # Open the PDF file
        reader = PdfReader(os.path.join(Directory, filename))
       
        # Append the pages to the new PDF file
        writer.addpages(reader.pages)

    # Create the cache directory if it doesn't exist
    os.makedirs(os.path.join(Directory, 'cache'), exist_ok=True)

    # Save the new PDF file
    writer.write(os.path.join(Directory, 'cache', 'combined.pdf'))
    
    # Get the end time
    end_time = time.time()

    # Print the running time
    print(f"Running time: {end_time - start_time} seconds")



combine_pdfs(Directory, Subjects_in_Headlines, Report_Type)

Combining PDFs: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.60it/s]

Running time: 0.03293466567993164 seconds


In [5]:
if __name__ == "__main__":

    start_time = time.time()
    filename = "cache/combined.pdf"
    filename = os.path.join(Directory, filename)

    doc = fitz.open(filename)
    total_pages = len(doc)
    cpu = min(cpu_count(),total_pages)
    # Close the PDF
    doc.close()
    # make vectors of arguments for the processes
    vectors = [(i, cpu, filename, C_dict) for i in range(cpu)]
    print("Starting %i processes for '%s'." % (cpu, filename))

    # Create a manager object
    manager = Manager()

    # Create a shared variable for the counter
    counter = manager.Value('i', 0)

    # Create a progress bar
    with Pool() as pool:
        results = []
        for result in pool.imap_unordered(process_file_wrapper, [(vector, counter) for vector in vectors]):
            results.append(result)
            # Update the progress bar here, not in the process_file function
    
    # Flatten the list of results
#    flat_results = [item for sublist in results for item in sublist]

    # Convert the results into a DataFrame
    # Convert the results into a DataFrame
    flat_results = [item for sublist in results for item in sublist]
    df = pd.DataFrame(flat_results)

    end_time = time.time()
    # Print the running time
    print(f"Running time: {end_time - start_time} seconds")

Starting 9 processes for 'C:/Hanbo Wang/LEAP/2023/ELA/t\cache/combined.pdf'.
Running time: 1.3836274147033691 seconds


In [8]:
df[df['LASID'].str.strip() == '9852238736']

,Page_Number,Advanced_School_Percentage,Advanced_School_System_Percentage,Advanced_State_Percentage,Mastery_School_Percentage,Mastery_School_System_Percentage,Mastery_State_Percentage,Basic_School_Percentage,Basic_School_System_Percentage,Basic_State_Percentage,Approaching_Basic_School_Percentage,Approaching_Basic_School_System_Percentage,Approaching_Basic_State_Percentage,Unsatisfactory_School_Percentage,Unsatisfactory_School_System_Percentage,Unsatisfactory_State_Percentage,Student_Performance_Score,If_Voided,Report_Title,Report_Subject,Report_Season_Year,Student_Name,Grade,LASID,School,School_System,DoB,School_System_Average_Score,State_Average_Score,Student_Performance_Level,School_System_Average_Level,State_Average_Level,Student_Performance_Achievement_Level,School_System_Average_Achievement_Level,State_Average_Achievement_Level_Fixed,Reading_Performance_Achievement_Level,Literary_Text_Achievement_Level,Informational_Text_Achievement_Level,Vocabulary_Achievement_Level,Reading_Performance_Achievement_Level_State_Percentages,Writing_Performance_Achievement_Level,Writing_Performance_Achievement_Level_State_Percentages,Written_Expression_Achievement_Level,Knowledge&Use_of_Language_Conventions
5,0,NR,N/A,15%,NR,N/A,27%,NR,N/A,25%,NR,N/A,18%,NR,N/A,15%,757,False,LEAP 2025,English Language Arts,Spring 2023,JAMIAH FELTON,7,9852238736,001 Thrive Academy,3C1 Thrive Academy,05/02/2009,,743,4,,3,MASTERY,,BASIC,STRONG\nPERFORMANCE,«««\nSTRONG\nPERFORMANCE,«««\nSTRONG\nPERFORMANCE,«««\nMODERATE\nPERFORMANCE,42%\n25%\n33%,«««\nMODERATE\nPERFORMANCE,53%\n15%\n31%,«««\nMODERATE\nPERFORMANCE,«««\nMODERATE\nPERFORMANCE


In [122]:
df2 = pd.DataFrame()

df2 = df.copy()

In [123]:
#Regulate the data in school column
df2['Student_Name'] = df2['Student_Name'].str.strip()
df2['School'] = df2['School'].str.strip()
df2['Grade'] = df2['Grade'].str.strip()
df2['LASID'] = df2['LASID'].str.strip()
df2['School'] = df2['School'].str.strip()
df2['School_System'] = df2['School_System'].str.strip()
df2['DoB'] = df2['DoB'].str.strip()
df2['Reading_Performance_Achievement_Level'] = df2['Reading_Performance_Achievement_Level'].str.replace('\n', ' ')
df2['Literary_Text_Achievement_Level'] = df2['Literary_Text_Achievement_Level'].str.replace('«««', '').str.replace('\n', ' ')
df2['Informational_Text_Achievement_Level'] = df2['Informational_Text_Achievement_Level'].str.replace('«««', '').str.replace('\n', ' ')
df2['Vocabulary_Achievement_Level'] = df2['Vocabulary_Achievement_Level'].str.replace('«««', '').str.replace('\n', ' ')
df2[['Reading_Performance_Achievement_Level_State_Percentage_Strong', 
     'Reading_Performance_Achievement_Level_State_Percentage_Moderate', 
     'Reading_Performance_Achievement_Level_State_Percentage_Weak']] = df2['Reading_Performance_Achievement_Level_State_Percentages'].str.strip().str.split('\n', expand=True)
df2['Writing_Performance_Achievement_Level'] = df2['Writing_Performance_Achievement_Level'].str.replace('«««', '').str.replace('\n', ' ')
df2[['Writing_Performance_Achievement_Level_State_Percentage_Strong', 
     'Writing_Performance_Achievement_Level_State_Percentage_Moderate', 
     'Writing_Performance_Achievement_Level_State_Percentage_Weak']] = df2['Writing_Performance_Achievement_Level_State_Percentages'].str.strip().str.split('\n', expand=True)
df2['Written_Expression_Achievement_Level'] = df2['Written_Expression_Achievement_Level'].str.replace('«««', '').str.replace('\n', ' ')
df2['Knowledge&Use_of_Language_Conventions'] = df2['Knowledge&Use_of_Language_Conventions'].str.replace('«««', '').str.replace('\n', ' ')



In [124]:
report_subjects = df2['Report_Subject'].unique()

if len(report_subjects) == 1:
    subject = report_subjects[0]

if subject == 'English Language Arts':
    subject = 'ELA'

subject

'ELA'

In [125]:
# Split the 'Student_Name' column into two new columns
df2[['Student_First_Name', 'Rest']] = df2['Student_Name'].str.split(' ', n=1, expand=True)

# Check if 'Rest' contains a space
contains_space = df2['Rest'].str.contains(' ')

# If 'Rest' contains a space, split it into 'Middle_Initial' and 'Last_Name'
df2.loc[contains_space, ['Student_Middle_Initial', 'Student_Last_Name']] = df2.loc[contains_space, 'Rest'].str.split(' ', n=1, expand=True)

# If 'Rest' doesn't contain a space, it only contains the last name
df2.loc[~contains_space, 'Student_Last_Name'] = df2.loc[~contains_space, 'Rest']
df2.loc[~contains_space, 'Student_Middle_Initial'] = np.nan

# Now you can drop the 'Rest' column as it's no longer needed
df2 = df2.drop(columns='Rest')


In [126]:
# Replace '--/--/----' with np.nan
df2['DoB'] = df2['DoB'].replace('--/--/----', np.nan)

# Convert the 'DoB' column to datetime format
df2['DoB'] = pd.to_datetime(df2['DoB'], errors='coerce')

# Create new columns for the day, month, and year
df2['Summarized_DOB_Day'] = df2['DoB'].dt.day
df2['Summarized_DOB_Month'] = df2['DoB'].dt.month
df2['Summarized_DOB_Year'] = df2['DoB'].dt.year

# Now you can drop the 'DoB' column as it's no longer needed
df2 = df2.drop(columns='DoB')

In [127]:
# Split the 'School' column into two new columns 'School_Code' and 'School_Name'
df2[['School_Code', 'School_Name']] = df2['School'].str.split(' ', n=1, expand=True)
df2 = df2.drop(columns='School')


df2[['School_System_Code', 'School_System_Name']] = df2['School_System'].str.split(' ', n=1, expand=True)
df2 = df2.drop(columns=['School_System', 'Reading_Performance_Achievement_Level_State_Percentages', 'Writing_Performance_Achievement_Level_State_Percentages'])

In [128]:
df2 = df2.rename(columns={'Grade': 'Summarized_Grade'})
df2 = df2.rename(columns={'Student_Performance_Score': f'Scale_Score_-_{subject}'})

In [130]:
df2

,Page_Number,Advanced_School_Percentage,Advanced_School_System_Percentage,Advanced_State_Percentage,Mastery_School_Percentage,Mastery_School_System_Percentage,Mastery_State_Percentage,Basic_School_Percentage,Basic_School_System_Percentage,Basic_State_Percentage,Approaching_Basic_School_Percentage,Approaching_Basic_School_System_Percentage,Approaching_Basic_State_Percentage,Unsatisfactory_School_Percentage,Unsatisfactory_School_System_Percentage,Unsatisfactory_State_Percentage,Scale_Score_-_ELA,If_Voided,Report_Title,Report_Subject,Report_Season_Year,Student_Name,Summarized_Grade,LASID,School_System_Average_Score,State_Average_Score,Student_Performance_Level,School_System_Average_Level,State_Average_Level,Student_Performance_Achievement_Level,School_System_Average_Achievement_Level,State_Average_Achievement_Level_Fixed,Reading_Performance_Achievement_Level,Literary_Text_Achievement_Level,Informational_Text_Achievement_Level,Vocabulary_Achievement_Level,Writing_Performance_Achievement_Level,Written_Expression_Achievement_Level,Knowledge&Use_of_Language_Conventions,State_Average_Achievement_Level,Reading_Performance_Achievement_Level_State_Percentage_Strong,Reading_Performance_Achievement_Level_State_Percentage_Moderate,Reading_Performance_Achievement_Level_State_Percentage_Weak,Writing_Performance_Achievement_Level_State_Percentage_Strong,Writing_Performance_Achievement_Level_State_Percentage_Moderate,Writing_Performance_Achievement_Level_State_Percentage_Weak,Student_First_Name,Student_Middle_Initial,Student_Last_Name,Summarized_DOB_Day,Summarized_DOB_Month,Summarized_DOB_Year,School_Code,School_Name,School_System_Code,School_System_Name
0,22256,7%,6%,6%,20%,37%,36%,22%,22%,20%,27%,18%,16%,24%,17%,22%,695,False,LEAP 2025,English Language Arts,Spring 2023,CHRISTIAN FRANCIS,3,7831378545,744,740,1,3,3,UNSATISFACTORY,SIC,BASIC,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,NaN,43%,20%,37%,43%,21%,37%,CHRISTIAN,NaN,FRANCIS,29.0,7.0,2013.0,071,Magnolia Elementary,023,Iberia Parish
1,22257,7%,6%,6%,20%,37%,36%,22%,22%,20%,27%,18%,16%,24%,17%,22%,685,False,LEAP 2025,English Language Arts,Spring 2023,MIGUEL GAGE,3,5412525873,744,740,1,3,3,UNSATISFACTORY,SIC,BASIC,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,NaN,43%,20%,37%,43%,21%,37%,MIGUEL,NaN,GAGE,12.0,2.0,2012.0,071,Magnolia Elementary,023,Iberia Parish
2,22258,7%,6%,6%,20%,37%,36%,22%,22%,20%,27%,18%,16%,24%,17%,22%,717,False,LEAP 2025,English Language Arts,Spring 2023,EZAVIEN GREEN,3,8828116684,744,740,2,3,3,APPROACHING BASIC,SIC,BASIC,MODERATE PERFORMANCE,MODERATE PERFORMANCE,WEAK PERFORMANCE,MODERATE PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,NaN,43%,20%,37%,43%,21%,37%,EZAVIEN,NaN,GREEN,10.0,8.0,2014.0,071,Magnolia Elementary,023,Iberia Parish
3,22259,7%,6%,6%,20%,37%,36%,22%,22%,20%,27%,18%,16%,24%,17%,22%,720,False,LEAP 2025,English Language Arts,Spring 2023,AYCETON HALL,3,1271283891,744,740,2,3,3,APPROACHING BASIC,SIC,BASIC,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,STRONG PERFORMANCE,MODERATE PERFORMANCE,MODERATE PERFORMANCE,WEAK PERFORMANCE,NaN,43%,20%,37%,43%,21%,37%,AYCETON,NaN,HALL,25.0,8.0,2014.0,071,Magnolia Elementary,023,Iberia Parish
4,22260,7%,6%,6%,20%,37%,36%,22%,22%,20%,27%,18%,16%,24%,17%,22%,685,False,LEAP 2025,English Language Arts,Spring 2023,ZAKHARIA HAWK,3,8934384727,744,740,1,3,3,UNSATISFACTORY,SIC,BASIC,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,MODERATE PERFORMANCE,NaN,43%,20%,37%,43%,21%,37%,ZAKHARIA,NaN,HAWK,19.0,7.0,2014.0,071,Magnolia Elementary,023,Iberia Parish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66762,66762,3%,8%,11%,30%,29%,37%

In [131]:
df2['Summarized_DOB_Year'].value_counts()

Summarized_DOB_Year
2009.0    11023
2010.0    10748
2011.0    10715
2012.0    10645
2013.0    10453
2014.0     6757
2008.0     5357
2007.0      975
2006.0       64
2015.0       12
1900.0        4
2005.0        4
2022.0        3
2020.0        1
2003.0        1
2004.0        1
2016.0        1
1988.0        1
Name: count, dtype: int64

In [132]:
df2[df2['LASID'] == '6761645433']

,Page_Number,Advanced_School_Percentage,Advanced_School_System_Percentage,Advanced_State_Percentage,Mastery_School_Percentage,Mastery_School_System_Percentage,Mastery_State_Percentage,Basic_School_Percentage,Basic_School_System_Percentage,Basic_State_Percentage,Approaching_Basic_School_Percentage,Approaching_Basic_School_System_Percentage,Approaching_Basic_State_Percentage,Unsatisfactory_School_Percentage,Unsatisfactory_School_System_Percentage,Unsatisfactory_State_Percentage,Scale_Score_-_ELA,If_Voided,Report_Title,Report_Subject,Report_Season_Year,Student_Name,Summarized_Grade,LASID,School_System_Average_Score,State_Average_Score,Student_Performance_Level,School_System_Average_Level,State_Average_Level,Student_Performance_Achievement_Level,School_System_Average_Achievement_Level,State_Average_Achievement_Level_Fixed,Reading_Performance_Achievement_Level,Literary_Text_Achievement_Level,Informational_Text_Achievement_Level,Vocabulary_Achievement_Level,Writing_Performance_Achievement_Level,Written_Expression_Achievement_Level,Knowledge&Use_of_Language_Conventions,State_Average_Achievement_Level,Reading_Performance_Achievement_Level_State_Percentage_Strong,Reading_Performance_Achievement_Level_State_Percentage_Moderate,Reading_Performance_Achievement_Level_State_Percentage_Weak,Writing_Performance_Achievement_Level_State_Percentage_Strong,Writing_Performance_Achievement_Level_State_Percentage_Moderate,Writing_Performance_Achievement_Level_State_Percentage_Weak,Student_First_Name,Student_Middle_Initial,Student_Last_Name,Summarized_DOB_Day,Summarized_DOB_Month,Summarized_DOB_Year,School_Code,School_Name,School_System_Code,School_System_Name
5156,27412,≤1%,6%,6%,10%,28%,36%,25%,18%,20%,25%,17%,16%,39%,31%,22%,673,False,LEAP 2025,English Language Arts,Spring 2023,KING ALLEN,3,6761645433,730,740,1,3,3,UNSATISFACTORY,SIC,BASIC,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,WEAK PERFORMANCE,NaN,43%,20%,37%,43%,21%,37%,KING,NaN,ALLEN,16.0,2.0,2013.0,030,Emmett C.Gilbert School of Excellence at,026,Jefferson Parish


In [133]:
df2.to_csv('C:/Hanbo Wang/Report_Readin_2023_ELA.csv')

def delete_file(filepath):
    # Check if the file exists
    if os.path.exists(filepath):
        # Delete the file
        os.remove(filepath)
        print(f"File {filepath} has been deleted.")
    else:
        print(f"File {filepath} does not exist.")



# specify the file to be deleted
filepath = os.path.join(Directory, 'cache', 'combined.pdf')

delete_file(filepath)

%%snakeviz
process_all_files()


### Don't run cells below

In [ ]:
df2 = pd.read_csv('C:/Hanbo Wang/Report_Readin_all_2022.csv', low_memory=False,index_col=0)

In [ ]:
df2

In [ ]:
df2['School_Code'].fillna('', inplace=True)
df2['School'] = df2['School_Code'].astype(str) + ' ' + df2['School_Name']

In [ ]:
# Split the 'School' column into two new columns 'School_Code' and 'School_Name'
df2[['School_Code', 'School_Name']] = df2['School'].str.split(' ', n=1, expand=True)
df2 = df2.drop(columns='School')


In [ ]:
df2['School_Code'].dtype

In [ ]:
df2[df2['LASID'].str.strip() == '6258331866']

In [ ]:
df2